In [2]:
import pandas as pd
import numpy as np
import subprocess
import glob
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
metadata = pd.read_csv("../filtered_DARPA_metadata.csv")
platemap = pd.read_csv("../huck_sub_id.csv")

In [4]:
metadata = platemap.merge(metadata, how = "left", on = "Huck_Sub_ID")

In [5]:
metadata["Novogene_ID"] = "A" + metadata["Database ID"].astype(str)

In [5]:
for i in metadata["Novogene_ID"].tolist():
    task = "symbiont_counts"
    mem = "16"
    cpus = "1"
    inpath = "~/scratch/darpa/mapped/"
    outpath = "../symbiont_counts/"
    prefix = i
    subprocess.run(["sbatch --mem=" + mem + "g --ntasks=" + 
         cpus + " ../" + task + ".sh " + 
         inpath + " " + outpath + " " + prefix], shell=True)

Submitted batch job 5084389
Submitted batch job 5084390
Submitted batch job 5084391
Submitted batch job 5084392
Submitted batch job 5084393
Submitted batch job 5084394
Submitted batch job 5084395
Submitted batch job 5084396
Submitted batch job 5084397
Submitted batch job 5084398
Submitted batch job 5084399
Submitted batch job 5084400
Submitted batch job 5084401
Submitted batch job 5084402
Submitted batch job 5084403
Submitted batch job 5084404
Submitted batch job 5084405
Submitted batch job 5084406
Submitted batch job 5084407
Submitted batch job 5084408
Submitted batch job 5084409
Submitted batch job 5084410
Submitted batch job 5084411
Submitted batch job 5084412
Submitted batch job 5084413
Submitted batch job 5084414
Submitted batch job 5084415
Submitted batch job 5084416
Submitted batch job 5084417
Submitted batch job 5084418
Submitted batch job 5084419
Submitted batch job 5084420
Submitted batch job 5084421
Submitted batch job 5084422
Submitted batch job 5084423
Submitted batch job 

In [6]:
sra = pd.read_csv("../DARPASraRunTable.txt")
sra = sra[~sra["Sample Name"].str.contains("Aprol")]

In [8]:
for i in sra["Run"].tolist():
    task = "symbiont_counts"
    mem = "16"
    cpus = "1"
    inpath = "~/scratch/darpa/mapped/"
    outpath = "../symbiont_counts/"
    prefix = i
    subprocess.run(["sbatch --mem=" + mem + "g --ntasks=" + 
         cpus + " ../" + task + ".sh " + 
         inpath + " " + outpath + " " + prefix], shell=True)

Submitted batch job 6808560
Submitted batch job 6808561
Submitted batch job 6808562
Submitted batch job 6808563
Submitted batch job 6808564
Submitted batch job 6808565
Submitted batch job 6808566
Submitted batch job 6808567
Submitted batch job 6808568
Submitted batch job 6808569
Submitted batch job 6808570
Submitted batch job 6808571
Submitted batch job 6808572
Submitted batch job 6808573
Submitted batch job 6808574
Submitted batch job 6808575
Submitted batch job 6808576
Submitted batch job 6808577
Submitted batch job 6808578
Submitted batch job 6808579
Submitted batch job 6808580
Submitted batch job 6808581
Submitted batch job 6808582
Submitted batch job 6808583
Submitted batch job 6808584
Submitted batch job 6808585
Submitted batch job 6808586
Submitted batch job 6808587


In [9]:
kenkel = pd.read_csv("../KenkelSequencingSampleKeyHaplotypeReferences.csv")

In [11]:
for i in kenkel["BaumsName"].tolist():
    task = "symbiont_counts"
    mem = "16"
    cpus = "1"
    inpath = "~/scratch/darpa/mapped/"
    outpath = "../symbiont_counts/"
    prefix = i
    subprocess.run(["sbatch --mem=" + mem + "g --ntasks=" + 
         cpus + " ../" + task + ".sh " + 
         inpath + " " + outpath + " " + prefix], shell=True)

Submitted batch job 6808588
Submitted batch job 6808589
Submitted batch job 6808590
Submitted batch job 6808591
Submitted batch job 6808592
Submitted batch job 6808593
Submitted batch job 6808594
Submitted batch job 6808595
Submitted batch job 6808596
Submitted batch job 6808597
Submitted batch job 6808598
Submitted batch job 6808599
Submitted batch job 6808600
Submitted batch job 6808601
Submitted batch job 6808602
Submitted batch job 6808603
Submitted batch job 6808604
Submitted batch job 6808605
Submitted batch job 6808606
Submitted batch job 6808607
Submitted batch job 6808608
Submitted batch job 6808609
Submitted batch job 6808610
Submitted batch job 6808611
Submitted batch job 6808612
Submitted batch job 6808613
Submitted batch job 6808614
Submitted batch job 6808615
Submitted batch job 6808616
Submitted batch job 6808617
Submitted batch job 6808618
Submitted batch job 6808619
Submitted batch job 6808620
Submitted batch job 6808621
Submitted batch job 6808622
Submitted batch job 

In [12]:
unpublished = glob.glob("/storage/home/nsl5160/scratch/darpa/unpublished_raw/*_1.fq.gz")

In [13]:
unpublished = [i.split("/")[7].split("_")[0] for i in unpublished]

In [14]:
for i in unpublished:
    task = "symbiont_counts"
    mem = "16"
    cpus = "1"
    inpath = "~/scratch/darpa/mapped/"
    outpath = "../symbiont_counts/"
    prefix = i
    subprocess.run(["sbatch --mem=" + mem + "g --ntasks=" + 
         cpus + " ../" + task + ".sh " + 
         inpath + " " + outpath + " " + prefix], shell=True)

Submitted batch job 6808628
Submitted batch job 6808629
Submitted batch job 6808630
Submitted batch job 6808631
Submitted batch job 6808632
Submitted batch job 6808633
Submitted batch job 6808634
Submitted batch job 6808635
Submitted batch job 6808636
Submitted batch job 6808637
Submitted batch job 6808638
Submitted batch job 6808639
Submitted batch job 6808640
Submitted batch job 6808641


In [25]:
symbiont_counts = pd.DataFrame(columns = ['symB', 'symC', 
                                        'symD', 'Apal', 
                                        'symA'])
for i in glob.glob("../symbiont_counts/*-mapped-sorted.txt"):
    tmp = pd.read_csv(i, sep = "\s", names = ["reads", "taxon"])
    tmp = pd.concat([tmp[tmp["taxon"].notna()], tmp[tmp["taxon"].isna()].tail(1)]).reset_index(drop=True)
    tmp.loc[tmp["taxon"].isna(), "taxon"] = "Total"
    tmp["reads"] = tmp.groupby(['taxon'])['reads'].transform('sum')
    tmp = tmp.drop_duplicates("taxon").reset_index(drop=True)
    tmp = tmp.set_index("taxon").T.reset_index(drop=True)
    tmp["Sample"] = i.split('/')[-1].split('-')[0]
    symbiont_counts = pd.concat([symbiont_counts, tmp]).reset_index(drop=True)

/tmp/ipykernel_2869281/831028233.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  tmp = pd.read_csv(i, sep = "\s", names = ["reads", "taxon"])
/tmp/ipykernel_2869281/831028233.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  tmp = pd.read_csv(i, sep = "\s", names = ["reads", "taxon"])
/tmp/ipykernel_2869281/831028233.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  tmp = pd.read_csv(i, sep = "\s", names

/tmp/ipykernel_2869281/831028233.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  tmp = pd.read_csv(i, sep = "\s", names = ["reads", "taxon"])
/tmp/ipykernel_2869281/831028233.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  tmp = pd.read_csv(i, sep = "\s", names = ["reads", "taxon"])
/tmp/ipykernel_2869281/831028233.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  tmp = pd.read_csv(i, sep = "\s", names

/tmp/ipykernel_2869281/831028233.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  tmp = pd.read_csv(i, sep = "\s", names = ["reads", "taxon"])
/tmp/ipykernel_2869281/831028233.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  tmp = pd.read_csv(i, sep = "\s", names = ["reads", "taxon"])
/tmp/ipykernel_2869281/831028233.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  tmp = pd.read_csv(i, sep = "\s", names

/tmp/ipykernel_2869281/831028233.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  tmp = pd.read_csv(i, sep = "\s", names = ["reads", "taxon"])
/tmp/ipykernel_2869281/831028233.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  tmp = pd.read_csv(i, sep = "\s", names = ["reads", "taxon"])
/tmp/ipykernel_2869281/831028233.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  tmp = pd.read_csv(i, sep = "\s", names

/tmp/ipykernel_2869281/831028233.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  tmp = pd.read_csv(i, sep = "\s", names = ["reads", "taxon"])
/tmp/ipykernel_2869281/831028233.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  tmp = pd.read_csv(i, sep = "\s", names = ["reads", "taxon"])
/tmp/ipykernel_2869281/831028233.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  tmp = pd.read_csv(i, sep = "\s", names

In [26]:
symbiont_counts["Symbiont_Reads"] = symbiont_counts["symB"] + symbiont_counts["symC"] + symbiont_counts["symD"] + symbiont_counts["symA"]
symbiont_counts["Perc_Symbiont"] = symbiont_counts["Symbiont_Reads"] / (symbiont_counts["Apal"] + symbiont_counts["Symbiont_Reads"])
symbiont_counts["Percent_Symbiodinium"] = symbiont_counts["symA"] / symbiont_counts["Symbiont_Reads"]
symbiont_counts["Percent_Breviolum"] = symbiont_counts["symB"] / symbiont_counts["Symbiont_Reads"]
symbiont_counts["Percent_Cladocopium"] = symbiont_counts["symC"] / symbiont_counts["Symbiont_Reads"]
symbiont_counts["Percent_Durusdinium"] = symbiont_counts["symD"] / symbiont_counts["Symbiont_Reads"]
symbiont_counts["Host_Reads"] = symbiont_counts["Apal"]

In [27]:
symbiont_counts.sort_values("Percent_Symbiodinium").head(60)

,symB,symC,symD,Apal,symA,Total,Sample,Symbiont_Reads,Perc_Symbiont,Percent_Symbiodinium,Percent_Breviolum,Percent_Cladocopium,Percent_Durusdinium,Host_Reads
112,14991,32152,17004949,112449647,417243,NaN,AP77_kenkel,17469335,0.134463,0.023884,0.000858,0.00184,0.973417,112449647
105,20907,42487,17833048,73067701,438802,123731807.0,AP327_kenkel,18335244,0.200598,0.023932,0.00114,0.002317,0.97261,73067701
109,11713,23775,7499713,82212934,189780,120538277.0,AP63_kenkel,7724981,0.085892,0.024567,0.001516,0.003078,0.970839,82212934
117,31152,70614,23558056,62065926,665934,118593538.0,AP91_kenkel,24325756,0.281575,0.027376,0.001281,0.002903,0.968441,62065926
107,51346,81912,20024970,70922443,567625,118747711.0,AP55_kenkel,20725853,0.226146,0.027387,0.002477,0.003952,0.966183,70922443
116,32587,80956,18680444,64408499,544718,NaN,AP8_kenkel,19338705,0.230918,0.028167,0.001685,0.004186,0.965961,64408499
102,26927,59073,19750876,43261200,595103,86241263.0,AP14_kenkel,20431979,0.320788,0.029126,0.001318,0.002891,0.966665,43261200
113,62793,131842,44499937,36656925,3789208,108939236.0,AP79_kenkel,48483780,0.569455,0.078154,0.001295,0.002719,0.917831,36656925
115,15634,21795,5398459,74861944,592178,117690910.0,AP89_kenkel,6028066,0.074522,0.098237,0.002594,0.003616,0.895554,74861944
110,49771,100125,29274469,64104109,4669953,136997987.0,AP66_kenkel,34094318,0.347198,0.136972,0.00146,0.002937,0.858632,64104109


In [20]:
symbiont_counts.sort_values("Perc_Symbiont")

,symB,symC,symD,Apal,symA,Total,Sample,Symbiont_Reads,Perc_Symbiont,Percent_Symbiodinium,Percent_Breviolum,Percent_Cladocopium,Percent_Durusdinium,Host_Reads
0,1560,2139,44,60926328,14460,66059949.0,A13414,18203,0.000299,0.794375,0.0857,0.117508,0.002417,60926328
66,811,1025,32,35253140,93688,38196066.0,A22151,95556,0.002703,0.980451,0.008487,0.010727,0.000335,35253140
49,974,1274,63,36525196,161725,39810520.0,A22125,164036,0.004471,0.985912,0.005938,0.007767,0.000384,36525196
11,958,1246,76,39483069,183093,42772652.0,A22051,185373,0.004673,0.9877,0.005168,0.006722,0.00041,39483069
42,1351,1634,530,45281835,239588,49343748.0,A22103,243103,0.00534,0.985541,0.005557,0.006721,0.00218,45281835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,4856,6054,753,35952922,2034471,42235725.0,A22243,2046134,0.053847,0.9943,0.002373,0.002959,0.000368,35952922
84,2297,4790,280,37051160,2146196,43501279.0,A22234,2153563,0.054931,0.996579,0.001067,0.002224,0.00013,37051160
88,3893,8198,395,45717881,3462743,53387275.0,A22239,3475229,0.070645,0.996407,0.00112,0.002359,0.000114,45717881
89,3544,6679,340,43485255,3712759,51111849.0,A22240,3723322,0.07887,0.997163,0.000952,0.001794,0.000091,43485255


The seq data agrees with the qPCR results they found in the lab. All of these are dominated by Symbiodinium. The only "exception" is the wild Dry Tortugas individual which is 79% Symbiodinium but that seems to just be because the overall number of symbiont reads is very low so the ability to bin them by genus is also low.